## Loading the dataset

In [ ]:
import gdown
import pandas as pd

file_id = '1zYOV1cMzJ6yCrPAKHJjCb2-nNeS6kUWt'
gdown.download(f'https://drive.google.com/uc?id={file_id}', 'healthcare-dataset-stroke-data.csv', quiet=False)

df = pd.read_csv('healthcare-dataset-stroke-data.csv')

df.head()

Downloading...
From: https://drive.google.com/uc?id=1zYOV1cMzJ6yCrPAKHJjCb2-nNeS6kUWt
To: /content/healthcare-dataset-stroke-data.csv
100%|██████████| 317k/317k [00:00<00:00, 52.6MB/s]


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


## Preprocessing

Check DataFrame structure, data types, and non-null counts

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


Check for missing values in the dataset

In [ ]:
df.isnull().sum()

,0
id,0
gender,0
age,0
hypertension,0
heart_disease,0
ever_married,0
work_type,0
Residence_type,0
avg_glucose_level,0
bmi,201


Fill missing bmi values with the median

In [ ]:
df['bmi'].fillna(df['bmi'].median(), inplace=True)

<ipython-input-4-597ac8b1dda2>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['bmi'].fillna(df['bmi'].median(), inplace=True)


Generate summary statistics for numerical columns

In [ ]:
df.describe()

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
count,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000
mean,36517.829354,43.226614,0.097456,0.054012,106.147677,28.862035,0.048728
std,21161.721625,22.612647,0.296607,0.226063,45.283560,7.699562,0.215320
min,67.000000,0.080000,0.000000,0.000000,55.120000,10.300000,0.000000
25%,17741.250000,25.000000,0.000000,0.000000,77.245000,23.800000,0.000000
50%,36932.000000,45.000000,0.000000,0.000000,91.885000,28.100000,0.000000
75%,54682.000000,61.000000,0.000000,0.000000,114.090000,32.800000,0.000000
max,72940.000000,82.000000,1.000000,1.000000,271.740000,97.600000,1.000000


Check unique values in the gender column

In [ ]:
df['gender'].value_counts()

,count
gender,
Female,2994
Male,2115
Other,1


Replace 'Other' in gender with the mode

In [ ]:
df['gender'] = df['gender'].replace({'Other': df['gender'].mode()[0]})

Verify value counts in gender after replacement

In [ ]:
df['gender'].value_counts()

,count
gender,
Female,2995
Male,2115


Count unique values in the other categorical features

In [ ]:
df['hypertension'].value_counts()

,count
hypertension,
0,4612
1,498


In [ ]:
df['heart_disease'].value_counts()

,count
heart_disease,
0,4834
1,276


In [ ]:
df['ever_married'].value_counts()

,count
ever_married,
Yes,3353
No,1757


In [ ]:
df['work_type'].value_counts()

,count
work_type,
Private,2925
Self-employed,819
children,687
Govt_job,657
Never_worked,22


In [ ]:
df['Residence_type'].value_counts()

,count
Residence_type,
Urban,2596
Rural,2514


In [ ]:
df.rename(columns={'Residence_type': 'residence_type'}, inplace=True)

In [ ]:
df['smoking_status'].value_counts()

,count
smoking_status,
never smoked,1892
Unknown,1544
formerly smoked,885
smokes,789


Drop the id column, not needed for analysis

In [ ]:
df.drop(columns=['id'], inplace=True)

Convert categorical columns to numeric codes for modeling

In [ ]:
categorical = ['gender', 'ever_married', 'work_type', 'residence_type', 'smoking_status']

for each in categorical:
  df[each] = df[each].astype('category')
  df[each] = df[each].cat.codes

Check if categorical features have been encoded correctly

In [ ]:
df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,67.0,0,1,1,2,1,228.69,36.6,1,1
1,0,61.0,0,0,1,3,0,202.21,28.1,2,1
2,1,80.0,0,1,1,2,0,105.92,32.5,2,1
3,0,49.0,0,0,1,2,1,171.23,34.4,3,1
4,0,79.0,1,0,1,3,0,174.12,24.0,2,1


Check class distribution of the target column stroke

In [ ]:
df['stroke'].value_counts()

,count
stroke,
0,4861
1,249


Apply SMOTE resampling to handle class distribution imbalance of the target

In [ ]:
from imblearn.over_sampling import SMOTE

X = df.drop('stroke', axis=1)
y = df['stroke']

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

print("Original class distribution:\n", y.value_counts())
print("\nResampled class distribution:\n", y_resampled.value_counts())

Original class distribution:
 stroke
0    4861
1     249
Name: count, dtype: int64

Resampled class distribution:
 stroke
1    4861
0    4861
Name: count, dtype: int64


## Gradient Boosting

Following the steps in
- https://datamapu.com/posts/classical_ml/gradient_boosting_classification_example/
- https://datamapu.com/posts/classical_ml/decision_tree_classification_example/

Define the functions

In [ ]:
import numpy as np

def sigmoid(x):
    """ Sigmoid function to convert log-odds to probabilities """
    return 1 / (1 + np.exp(-x)) # Converts log-odds to probabilities between 0 and 1

def log_loss(y, p):
    """ Log loss function """
    return -np.mean(y * np.log(p) + (1 - y) * np.log(1 - p)) # Calculates how well the model's predictions match the actual labels

def best_split(X, residuals):
    """ Find the best feature and threshold to split the data based on residuals """
    best_split = None
    best_loss = float('inf')

    for feature_index in range(X.shape[1]):  # Iterate over all features
        thresholds = np.unique(X[:, feature_index])
        for threshold in thresholds:  # Iterate over all thresholds
            left_indices = X[:, feature_index] <= threshold
            right_indices = X[:, feature_index] > threshold

            if len(residuals[left_indices]) == 0 or len(residuals[right_indices]) == 0:
                continue

            left_residuals = residuals[left_indices]
            right_residuals = residuals[right_indices]

            # Calculate the weighted loss based on variance of residuals
            weighted_loss = len(left_residuals) * np.var(left_residuals) + len(right_residuals) * np.var(right_residuals)

            # Update the best split if the current split has a lower loss
            if weighted_loss < best_loss:
                best_loss = weighted_loss
                best_split = (feature_index, threshold)

    return best_split

def split(X, residuals, feature_index, threshold):
    """ Split the data based on feature and threshold """
    left_indices = X[:, feature_index] <= threshold
    right_indices = X[:, feature_index] > threshold
    return X[left_indices], residuals[left_indices], X[right_indices], residuals[right_indices]

def build_tree(X, residuals, depth=0, max_depth=None):
    """ Recursive function to build a decision tree for gradient boosting """
    if len(np.unique(residuals)) == 1 or (max_depth and depth >= max_depth):
        return np.mean(residuals)  # Return the mean of residuals

    # Find the best feature and threshold to split the data
    best_split_info = best_split(X, residuals)
    if best_split_info is None:
        return np.mean(residuals)

    # Get the feature index and threshold from the best split
    feature_index, threshold = best_split_info

    # Split the data into left and right branches
    left_X, left_residuals, right_X, right_residuals = split(X, residuals, feature_index, threshold)

    # Recursively build the left and right branches
    left_branch = build_tree(left_X, left_residuals, depth + 1, max_depth)
    right_branch = build_tree(right_X, right_residuals, depth + 1, max_depth)

    return (feature_index, threshold, left_branch, right_branch)

def predict(X, tree):
    """ Predict class labels (probabilities) for the input data using the tree """
    predictions = []
    for x in X:
        current_node = tree # Start at the root of the tree
        while isinstance(current_node, tuple):  # Traverse the tree until a leaf is reached
            feature_index, threshold, left_branch, right_branch = current_node
            # Choose left or right branch based on the feature value for the current data point
            if x[feature_index] <= threshold:
                current_node = left_branch
            else:
                current_node = right_branch
        # The leaf node gives the prediction
        predictions.append(current_node)
    return np.array(predictions)

def fit(X, y, n_estimators=100, learning_rate=0.1, max_depth=3):
    """ Fit the Gradient Boosting model to the data """
    p = np.mean(y) # Start with the mean of the target values
    initial_prediction = np.log(p / (1 - p)) # Convert the mean to log-odds for classification
    predictions = np.full(len(y), initial_prediction)  # Initialize predictions with log-odds

    trees = []

    for _ in range(n_estimators):
        residuals = y - sigmoid(predictions)  # Calculate residuals between actual values and model's current predictions

        # Train a tree on the residuals
        tree = build_tree(X, residuals, max_depth=max_depth)
        trees.append(tree)

        # Use the tree's predictions to update the model's predictions
        tree_predictions = predict(X, tree)
        predictions += learning_rate * tree_predictions  # Update log-odds predictions

    return trees, predictions

def gradient_boosting_classifier(X_train, y_train, X_test, n_estimators=100, learning_rate=0.1, max_depth=3):
    """ Train Gradient Boosting model and evaluate it """
    # Train the model on the training set
    trees, train_predictions = fit(X_train, y_train, n_estimators, learning_rate, max_depth)
    # Convert log-odds predictions to probabilities
    train_probabilities = sigmoid(train_predictions)
    # Convert probabilities to class labels based on a threshold of 0.5
    train_class_predictions = (train_probabilities >= 0.5).astype(int)

    # Evaluate on test data
    test_predictions_log_odds = np.zeros(len(X_test))
    for tree in trees:
        test_predictions_log_odds += learning_rate * predict(X_test, tree)
    # Convert the final log-odds predictions to probabilities
    test_probabilities = sigmoid(test_predictions_log_odds)
    # Convert probabilties to class labels based on a threshold of 0.5
    test_class_predictions = (test_probabilities >= 0.5).astype(int)

    # Return predictions for both training and test sets
    return train_probabilities, train_class_predictions, test_probabilities, test_class_predictions

In [ ]:
def accuracy(y_true, y_pred):
    """ Calculate accuracy of predictions """
    return np.mean(y_true == y_pred)

def precision(y_true, y_pred):
    """ Calculate precision of predictions """
    true_positives = np.sum((y_true == 1) & (y_pred == 1))
    predicted_positives = np.sum(y_pred == 1)
    return true_positives / predicted_positives if predicted_positives > 0 else 0

def recall(y_true, y_pred):
    """ Calculate recall of predictions """
    true_positives = np.sum((y_true == 1) & (y_pred == 1))
    actual_positives = np.sum(y_true == 1)
    return true_positives / actual_positives if actual_positives > 0 else 0

def f1_score(y_true, y_pred):
    """ Calculate F1 score of predictions """
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2 * (prec * rec) / (prec + rec) if (prec + rec) > 0 else 0

def confusion_matrix(y_true, y_pred):
    """ Calculate confusion matrix """
    tp = np.sum((y_true == 1) & (y_pred == 1))
    tn = np.sum((y_true == 0) & (y_pred == 0))
    fp = np.sum((y_true == 0) & (y_pred == 1))
    fn = np.sum((y_true == 1) & (y_pred == 0))
    return np.array([[tn, fp], [fn, tp]])

Split the data into train and test sets

In [ ]:
X = X_resampled.values
y = y_resampled.values

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Apply the gradient boosting classifier algorithm to the data

In [ ]:
train_prob, train_pred, test_prob, test_pred = gradient_boosting_classifier(X_train, y_train, X_test, n_estimators=20, learning_rate=0.1, max_depth=3)

Compare train and test accuracy scores and inspect predictions

In [ ]:
print("Train Accuracy:", accuracy(y_train, train_pred))
print("Test Accuracy:", accuracy(y_test, test_pred))
print("")
print("Test Probabilities:", test_prob)
print("Test Predictions:", test_pred)

Train Accuracy: 0.804937636620805
Test Accuracy: 0.7866323907455013

Test Probabilities: [0.31885653 0.64079571 0.57015025 ... 0.64079571 0.32089364 0.6181996 ]
Test Predictions: [0 1 1 ... 1 0 1]


Evaluate model performance on the test set

In [ ]:
acc = accuracy(y_test, test_pred)
prec = precision(y_test, test_pred)
rec = recall(y_test, test_pred)
f1 = f1_score(y_test, test_pred)
conf_matrix = confusion_matrix(y_test, test_pred)

print("Accuracy:", acc)
print("Precision:", prec)
print("Recall:", rec)
print("F1 Score:", f1)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.7866323907455013
Precision: 0.7615457115928369
Recall: 0.8329896907216495
F1 Score: 0.7956671590349582
Confusion Matrix:
 [[722 253]
 [162 808]]


**DESCRIPTION**

This code implements Gradient Boosting algorithm for classification where:
- Decision trees are the base learners.
- Each tree is trained on the residuals of the previous predictions.
- The predictions of the trees are additively combined to make the final prediction, with each tree's contribution scaled by a learning rate.
- Gradient Boosting minimizes a loss function at each iteration.

Generally, Gradient Boosting Method is then an algorithm that gradually increases its accuracy and minimizes prediction errors.

**STEPS**

1. Split the data into training and test sets.

2. Initialize the model with an initial prediction (log-odds for classification).

3. Fit trees on the residuals: Each tree learns to predict the error (difference) between the current predictions and the actual target labels.

4. Update predictions: After each tree is built, update the model's predictions by adding the tree's output, scaled by the learning rate to control how much it adjusts the predictions.

5. Repeat: This process is repeated for a set number of trees, with each tree improving the model's predictions.

6. Evaluate on the test set: After the model is trained on the training set, apply it to the test set to generate predictions.

6. Final prediction: This is obtained by applying the sigmoid function to the cumulative log-odds of all tree predictions.

**DIFFERENCE FROM RANDOM FORESTS**

In Random Forests, many decision trees are built independently. Each tree is trained on a random subset of the data. Once all the trees are built, the final prediction is made by averaging the results (for regression) or voting on the most common prediction (for classification).

In contrast, Gradient Boosting builds trees sequentially, or one after another. Each tree tries to fix the mistakes made by the previous tree. The idea is that each new tree focuses on improving the model by correcting the errors of the previous trees. This makes Gradient Boosting's computational speed slower because trees are built sequentially, with each tree depending on the residuals of the previous tree. However, it performs better in scenarios where high accuracy is needed.

Therefore, while gradient boosting builds each new model sequentially to reduce errors, random forests focus on training an ensemble of models independently or averages across a number of independent trees.